

<h1 align=center><font size = 5>Peer-graded Assignment: Build a Regression Model in Keras</font></h1>


<a id='item41'></a>


### Import Keras and Packages


In [15]:
#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6

In [16]:
import pandas as pd
import numpy as np

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

<a id='item42'></a>


### Import Dataset

In [18]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


# Part A - Baseline Model

In [19]:
# Split data into predictors and target
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

n_cols = predictors.shape[1] # number of predictors

In [20]:
# Splitting Train / Test Data
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)


In [21]:
# define regression model
def baseline():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [22]:
# build the model
model = baseline()

In [23]:
# fit the model
model.fit(x_train, y_train,  epochs=50, verbose=0)

In [24]:
# model predictions on test sample
y_pred = model.predict(x_test)


In [25]:
# Calculate MSE
from sklearn.metrics import mean_squared_error

mse = np.zeros(50)
mse[0] = mean_squared_error(y_test, y_pred)

### Loop

In [26]:
for i in range(0,50):
    # Splitting Train/Test Data
    seed = 42 + i*2
    x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=seed)
    # Model 
    model.fit(x_train, y_train,  epochs=50, verbose=0)
    # Evaluation
    y_pred = model.predict(x_test)
    mse[i] = mean_squared_error(y_test, y_pred)
    print("{}: Model MSE = {}".format(i,mse[i]))
    


0: Model MSE = 175.8046108572226
1: Model MSE = 132.82505549740722
2: Model MSE = 99.6192840669869
3: Model MSE = 77.3437439939325
4: Model MSE = 87.69575579391069
5: Model MSE = 74.5838056446534
6: Model MSE = 80.19399690546658
7: Model MSE = 65.03394902060134
8: Model MSE = 80.99423021059306
9: Model MSE = 48.935326488421516
10: Model MSE = 59.104836674801795
11: Model MSE = 53.88910065757842
12: Model MSE = 48.863890842464194
13: Model MSE = 43.35599234843158
14: Model MSE = 50.12138839587418
15: Model MSE = 48.79583677550232
16: Model MSE = 52.871449069951204
17: Model MSE = 44.32231896895998
18: Model MSE = 53.44056839724938
19: Model MSE = 47.62589261615415
20: Model MSE = 48.84923240372838
21: Model MSE = 48.46692406495215
22: Model MSE = 55.5020924669549
23: Model MSE = 49.02544449593439
24: Model MSE = 47.625886429721504
25: Model MSE = 52.02369606723581
26: Model MSE = 45.9374374777685
27: Model MSE = 48.79706482475827
28: Model MSE = 43.11839284047385
29: Model MSE = 49.5415

### Results A

In [27]:
a_mean_mse = np.mean(mse)
a_std_mse = np.std(mse)

df_results = pd.DataFrame.from_dict({"Part": ["A"],"mean_mse": [a_mean_mse], "std_sq_mse": [a_std_mse]})
df_results

,Part,mean_mse,std_sq_mse
0,A,56.143991,24.232888


# Part B - Normalized Data

In [28]:
# Normalizing
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


### Loop

In [29]:
for i in range(0,50):
    # Splitting Train/Test Data
    seed = 48 + i*2
    x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=seed)
    # Model 
    model.fit(x_train, y_train,  epochs=50, verbose=0)
    # Evaluation
    y_pred = model.predict(x_test)
    mse[i] = mean_squared_error(y_test, y_pred)
    print("{}: Model MSE = {}".format(i,mse[i]))

0: Model MSE = 404.0897896009085
1: Model MSE = 190.40768974264526
2: Model MSE = 116.83182192918429
3: Model MSE = 91.51402166907641
4: Model MSE = 61.340466808128696
5: Model MSE = 63.01140923056655
6: Model MSE = 44.86475241614612
7: Model MSE = 43.828523621763864
8: Model MSE = 42.51549010056815
9: Model MSE = 36.93405623981028
10: Model MSE = 37.2522999035918
11: Model MSE = 41.10469556496234
12: Model MSE = 35.55344734330195
13: Model MSE = 36.56508117294197
14: Model MSE = 35.70417671075946
15: Model MSE = 37.850595087269774
16: Model MSE = 37.67280980491196
17: Model MSE = 37.14514452563416
18: Model MSE = 32.17737068146608
19: Model MSE = 33.65294357647105
20: Model MSE = 34.673003319230695
21: Model MSE = 35.200880323601986
22: Model MSE = 36.77933387325709
23: Model MSE = 36.31579376238663
24: Model MSE = 36.47999810572271
25: Model MSE = 33.975524089871
26: Model MSE = 41.12460318943618
27: Model MSE = 34.01723637113123
28: Model MSE = 35.1813009687615
29: Model MSE = 36.26

### Results B 

In [30]:
b_mean_mse = np.mean(mse)
b_std_mse = np.std(mse)

df_results.loc[df_results.shape[0]] = ["B",b_mean_mse,b_std_mse]
df_results



,Part,mean_mse,std_sq_mse
0,A,56.143991,24.232888
1,B,49.433851,56.912651


# Part C - 100 Epochs

### Loop

In [31]:
for i in range(0,50):
    # Splitting Train/Test Data
    seed = 48 + i*2
    x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=seed)
    # Model 
    model.fit(x_train, y_train,  epochs=100, verbose=0)
    # Evaluation
    y_pred = model.predict(x_test)
    mse[i] = mean_squared_error(y_test, y_pred)
    print("{}: Model MSE = {}".format(i,mse[i]))

0: Model MSE = 33.80085554059416
1: Model MSE = 27.2704169570423
2: Model MSE = 27.857418879206936
3: Model MSE = 36.01896796301359
4: Model MSE = 28.338430391110755
5: Model MSE = 27.291053754422435
6: Model MSE = 28.461346060508756
7: Model MSE = 28.114186235678986
8: Model MSE = 28.881360823930038
9: Model MSE = 24.59779505513606
10: Model MSE = 27.712834286274052
11: Model MSE = 25.52810258071672
12: Model MSE = 25.815214027538342
13: Model MSE = 26.586908646033926
14: Model MSE = 24.365502793894382
15: Model MSE = 30.003880895168408
16: Model MSE = 26.068460409858506
17: Model MSE = 28.029581156940534
18: Model MSE = 24.370621340364597
19: Model MSE = 25.220768505191614
20: Model MSE = 23.48957246586129
21: Model MSE = 27.260020869246812
22: Model MSE = 23.397889536397816
23: Model MSE = 26.70484857415346
24: Model MSE = 26.684931868413777
25: Model MSE = 24.63429238927072
26: Model MSE = 29.96152040385726
27: Model MSE = 23.416398895338926
28: Model MSE = 25.76647596519868
29: Mo

### Results C

In [32]:
c_mean_mse = np.mean(mse)
c_std_mse = np.std(mse)

df_results.loc[df_results.shape[0]] = ["C",c_mean_mse,c_std_mse]
df_results



,Part,mean_mse,std_sq_mse
0,A,56.143991,24.232888
1,B,49.433851,56.912651
2,C,26.269451,2.770411


# Part D - Three Hidden Layers

### Model

In [33]:
def advanced():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Build the Model
model = advanced()

### Loop

In [34]:
for i in range(0,50):
    # Splitting Train/Test Data
    seed = 48 + i*2
    x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=seed)
    # Model 
    model.fit(x_train, y_train,  epochs=50, verbose=0)
    # Evaluation
    y_pred = model.predict(x_test)
    mse[i] = mean_squared_error(y_test, y_pred)
    print("{}: Model MSE = {}".format(i,mse[i]))

0: Model MSE = 125.06533206695788
1: Model MSE = 111.39317463096569
2: Model MSE = 98.3616439635067
3: Model MSE = 77.70997623351268
4: Model MSE = 44.53314899630184
5: Model MSE = 45.094452437857186
6: Model MSE = 40.90690744882608
7: Model MSE = 42.68983019598237
8: Model MSE = 39.34879768876064
9: Model MSE = 32.882680925260466
10: Model MSE = 29.64427684324661
11: Model MSE = 30.938585791051583
12: Model MSE = 28.97229964714618
13: Model MSE = 28.889544526981418
14: Model MSE = 28.124045450019786
15: Model MSE = 32.68567158705942
16: Model MSE = 26.2046706378604
17: Model MSE = 28.01648037741208
18: Model MSE = 24.90651351790487
19: Model MSE = 28.358453765701984
20: Model MSE = 29.662776559592935
21: Model MSE = 27.450869506084192
22: Model MSE = 28.012650246336555
23: Model MSE = 26.766643565960763
24: Model MSE = 27.823157870526398
25: Model MSE = 26.40365487212751
26: Model MSE = 29.816368890277715
27: Model MSE = 26.352299546754985
28: Model MSE = 27.896735325241195
29: Model 

### Results D 

In [35]:
d_mean_mse = np.mean(mse)
d_std_mse = np.std(mse)

df_results.loc[df_results.shape[0]] = ["D",d_mean_mse,d_std_mse]
df_results


,Part,mean_mse,std_sq_mse
0,A,56.143991,24.232888
1,B,49.433851,56.912651
2,C,26.269451,2.770411
3,D,34.092174,21.692680


<hr>

Copyright © 2019 Moritz Abe